# Geog 80 Final - Ryan Meyer
## COVID-19 Pandemic in the United States

### Section 1: Data loading and processing
First, we load in the covid case data, US Census 2019 population estimates, as well as some GeoJSON data containing county geometry. Some cleaning needs to be done to match the format of FIPS codes between datasets. Additionally, population dataset needs to be cleaned, then joined via census area names. Finally, after dividing the joined table by the data from the US population table, we yield the final dataset: a dataframe of covid cases per capita for each county for each day. This will be used to look at how COVID-19 spread across the United States over the course of 2020.

In [11]:
import pandas as pd
import json
from urllib.request import urlopen
import warnings

caseDataFile = "https://raw.githubusercontent.com/ryanpmeyer/COVID-19-Case-Timeline-Map/master/dashboard/time_series_covid19_confirmed_US.csv"
populationDataFile = "https://raw.githubusercontent.com/ryanpmeyer/COVID-19-Case-Timeline-Map/master/dashboard/co-est2019-alldata.csv"
geoJSONFile = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
eventsFile = 'https://raw.githubusercontent.com/ryanpmeyer/COVID-19-Case-Timeline-Map/master/dashboard/EVENTS.csv'

def getData():
  with urlopen(geoJSONFile) as response:
    counties = json.load(response)

  eventData = pd.read_csv(eventsFile)

  # get covid case data, clean FIPS code
  covid_cases_raw = pd.read_csv(caseDataFile)
  covid_cases = covid_cases_raw.dropna().reindex()
  old_fips = covid_cases['FIPS'].astype('int32').astype(str)
  max_fips_code_len = max([len(x) for x in old_fips])
  new_fips = list(old_fips)
  for i in range(len(old_fips)):
      while len(new_fips[i]) < max_fips_code_len:
          new_fips[i] = '0' + new_fips[i]
  covid_cases['FIPS'] = new_fips

  # get US population data, clean county names
  us_pop_raw = pd.read_csv(populationDataFile, encoding='latin1')
  us_pop_raw['CTY_OG'] = us_pop_raw['CTYNAME']
  us_pop = us_pop_raw[us_pop_raw['CTYNAME'] != us_pop_raw['STNAME']]
  strings2remove = [' County',' Borough',' Census Area',' Municipality',' Parish', ' city', ' City']
  for string in strings2remove:
      us_pop['CTYNAME'] = us_pop['CTYNAME'].str.replace(string,"",case=False)
  us_pop['CTYNAME'] = us_pop['CTYNAME'].str.strip()

  # remove data from population with duplicate locations
  indicies_to_remove = []
  for state in set(us_pop['STNAME']):
    counties_in_state = us_pop[us_pop['STNAME']==state]
    for cty in set(counties_in_state['CTYNAME']):
      rows = counties_in_state[counties_in_state['CTYNAME']==cty]
      if len(rows) > 1:
        rows_index = rows.index
        highest_pop_i = rows['POPESTIMATE2019'].argmax()
        for i in rows_index:
          if i != highest_pop_i:
            indicies_to_remove.append(i)
  us_pop.drop(index=indicies_to_remove,inplace=True)
  us_pop.reindex(copy=False)

  # join tables and divide case nums by population
  us_pop['place'] = us_pop['CTYNAME'] + ',' + us_pop['STNAME']
  covid_cases['place'] = covid_cases['Admin2'] + ',' + covid_cases['Province_State']
  joined = covid_cases.set_index('place').join(us_pop.set_index("place")[['POPESTIMATE2019','CTY_OG']],
              how='inner', lsuffix='County', rsuffix='').sort_index()

  # set county names back to original
  joined['Area_Name'] = joined['CTY_OG'] + ', '+joined['Province_State']
    
  # get case data per 10000 people
  for col in joined.columns[11:len(joined.columns)-2]:
      joined[col] = joined[col] / joined['POPESTIMATE2019'] * 10000 # per 10000 people



  used_fips = set(joined['FIPS'])
  # remove data from GeoJSON without matching FIPS code in case data
  
  cleaned_features = list(counties['features'])
  for feature in counties['features']:
    if feature['id'] not in used_fips:
      cleaned_features.remove(feature)
  counties['features'] = cleaned_features

  return counties, joined, eventData

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    counties, caseData, eventData = getData()
display(caseData.head(5))
display(eventData.head(5))

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,POPESTIMATE2019,CTY_OG,Area_Name
place,,,,,,,,,,,,,,,,,,,,,
"Abbeville,South Carolina",84045001,US,USA,840,45001,Abbeville,South Carolina,US,34.223334,-82.461707,...,380.804827,386.512823,391.405390,391.813104,394.259388,396.297957,398.744241,10000.0,Abbeville County,"Abbeville County, South Carolina"
"Acadia,Louisiana",84022001,US,USA,840,22001,Acadia,Louisiana,US,30.295065,-92.414197,...,625.997260,636.634701,636.634701,641.792248,641.147554,649.367395,657.426062,10000.0,Acadia Parish,"Acadia Parish, Louisiana"
"Accomack,Virginia",84051001,US,USA,840,51001,Accomack,Virginia,US,37.767072,-75.632346,...,409.704171,411.560837,414.655279,416.202500,416.511945,419.915831,420.534720,10000.0,Accomack County,"Accomack County, Virginia"
"Ada,Idaho",84016001,US,USA,840,16001,Ada,Idaho,US,43.452658,-116.241552,...,505.204667,515.400125,521.837176,526.675346,533.673043,542.809503,550.762375,10000.0,Ada County,"Ada County, Idaho"
"Adair,Iowa",84019001,US,USA,840,19001,Adair,Iowa,US,41.330756,-94.471059,...,599.832215,599.832215,615.212528,616.610738,616.610738,620.805369,626.398210,10000.0,Adair County,"Adair County, Iowa"


,DATE,LAT,LON,HEADLINE,DESCRIPTION,LINK
0,1/22/20,47.442224,-122.302510,First Travel-related Case of 2019 Novel Corona...,This first instance of a 2019-nCoV infected re...,https://www.cdc.gov/media/releases/2020/p0121-...
1,1/31/20,0.000000,0.000000,WHO Issues Global Health Emergency,As case numbers quickly began to jump exponent...,https://www.ajmc.com/view/what-were-reading-gl...
2,2/3/20,0.000000,0.000000,US Declares Public Health Emergency,"3 days after the WHO, the Trump administration...",https://www.jurist.org/news/2020/02/trump-admi...
3,2/25/20,0.000000,0.000000,CDC Says COVID-19 Is Heading toward Pandemic S...,Pandemics have 3 main factors: illness resulti...,https://www.cdc.gov/media/releases/2020/t0225-...
4,2/26/20,42.359896,-71.051103,Biogen meeting leads to COVID-19 spread in mul...,After a February meeting with senior executive...,https://www.nytimes.com/2020/12/11/us/biogen-c...


### Section 2: Choropleth Map

Now we have data for each US county's confirmed cases per capita for each day from 1/22 - 12/2. Next we will look at a choropleth map of this data. The following code constructs an interactive map and creates widgets which allow the data to be explored chronologically.

In [12]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipyleaflet
import ipywidgets as widgets
import datetime
import branca
import numpy as np

absolute = False
NUMDAYS = 315
STARTDATE = datetime.date(2020,1,22)
date = STARTDATE
old_date = None
event_widget = None
event_marker = None
choropleth_layer = None
legend_control = ipyleaflet.LegendControl({}, name = 'cases/10k residents', position = 'bottomright')
date_picker = None
hover_popup = None
data_series = caseData['1/22/20']
DATES = [STARTDATE + datetime.timedelta(days=x) for x in range(NUMDAYS)]
max_caseload = max(caseData['12/2/20'])
NUMEVENTS = len(eventData['DATE'])



def get_colorscale(max_val=max_caseload):
  return branca.colormap.LinearColormap(((144, 201, 222),
                                         (214, 57, 26)),
                                        vmin=0,vmax=max_val)

def get_date_string(date):
  date_string = date.strftime("%x") # format date into D/M/Y
  if date_string[0] == '0': # no leading zeros in month
    date_string = date_string[1:]
  if date_string[date_string.find('/')+1] == '0': # no leading zeros in day
    date_string = date_string[0:date_string.find('/')+1] + date_string[date_string.find('/')+2:]
  return date_string

def get_date_from_string(string):
    year = 2020
    month = int(string[:string.find('/')])
    day = string[string.find('/')+1:]
    day = int(day[:day.find('/')])
    return datetime.date(year,month,day)

def get_styler():
    global data_series, absolute
    def style(feature):
      max_val = max_caseload if absolute else max(data_series)
      colorscale = get_colorscale(max_val)
      value = data_series.get(feature['id'])
      if value is None:
        value = -1
      return {
          'color': 'black',
          'fillColor': colorscale(value) if value != -1 else 'black'
      }
    return style

def get_legend():
    global data_series, absolute
    max_val = max_caseload if absolute else max(data_series)
    colorscale = get_colorscale(max_val)
    legend_dict = {str(round(val,4)):colorscale(val) for val in np.linspace(0,max_val,6)}
    return legend_dict

def next_button(b):
    tomorrow = date+datetime.timedelta(days=1)
    next_date = None
    for event_date_str in eventData['DATE']:
        event_date = get_date_from_string(event_date_str)
        if event_date >= tomorrow:
            next_date = event_date
            break
    if not next_date:
        return
    update(event_date)

def prev_button(b):
    prev_date = date
    for event_date_str in eventData['DATE']:
        event_date = get_date_from_string(event_date_str)
        if event_date < prev_date:
            prev_date = event_date
    if not prev_date == date:
        return
    update(prev_date)
    
def hover_handler(event=None, feature=None, id=None, properties=None):
    case_val = round(data_series[id], 4)
    hover_popup.value = '<b>'+caseData.set_index('FIPS')['Area_Name'][id]+'</b>: '+str(case_val)
    
def update_event(new_date):
    global event_widget,event_marker
    events = eventData[eventData['DATE']==new_date]
    event_widget.value = ''
    if len(events):
        for event in [events.loc[i] for i in events.index]:
            event_marker.location = (event['LAT'],event['LON'])
            event_widget.value += (
                "<h1><b>"+event['HEADLINE']+"</b>:<br></h1>"+
                event['DESCRIPTION']+"<br>"+
                "<a href=\'"+event['LINK']+"\'>[SOURCE]</a><br>"
            )
    else:
        event_marker.location = (0,0)
        event_widget.value = ''


def get_choropleth(start_date):
  global absolute,date,old_date,hover_popup,event_widget,event_marker,choropleth_layer,date_picker,data_series
  old_date = start_date
  curr_event = 0
  date_string = get_date_string(date)
  data_series = caseData.set_index('FIPS')[date_string]

  m = ipyleaflet.Map(center=[48,-102],zoom=4)

  choropleth_layer = ipyleaflet.GeoJSON(
      data=counties,
      style={
          'opacity': 1.,
          'fillOpacity': 0.5,
          'weight': .3
      },
      hover_style={
          'fillOpacity': 1.,
          'weight': .6
      },
      style_callback=get_styler()
  )
  m.add_layer(choropleth_layer)

  # UI changes upon hover action
  hover_popup = widgets.HTML()
  hover_popup.layout.margin = '0px 20px 20px 20px'
  m.add_control(ipyleaflet.WidgetControl(widget=hover_popup,position='bottomleft'))
  choropleth_layer.on_hover(hover_handler)

  date_picker = widgets.DatePicker(value=STARTDATE, enabled_dates=DATES, description='Date: ',disabled=True)
  date_control = ipyleaflet.WidgetControl(widget=date_picker,position='topright')
  m.add_control(date_control)

  next_date_button = widgets.Button(description='Next Event')
  prev_date_button = widgets.Button(description='Prev Event')
  next_date_button.on_click(next_button)
  display(next_date_button)
  prev_date_button.on_click(prev_button)
  display(prev_date_button)
    
  # Markers for event information ("essay" part)
  event_marker = ipyleaflet.Marker()
  event_marker.visibile = False
  m.add_layer(event_marker)
  event_widget = widgets.HTML()
  event_widget.layout.margin =  '0px 20px 20px 20px'
    
  # legend
  legend_control.legends = get_legend()
  m.add_control(legend_control)

  return m

def update(new_date):
    global date,old_date,date_picker,hover_popup,choropleth_layer,data_series,legend_control
    if new_date not in DATES:
        date_picker.value = old_date
        return;
    date_picker.value = new_date
    old_date = date
    date = new_date
    date_string = get_date_string(date)
    hover_popup.value = "Updating map to: "+ date_string
    data_series = caseData.set_index('FIPS')[date_string]
    legend_control.legends = get_legend() # update legend
    choropleth_layer.style_callback = get_styler() # update style (colors)
    update_event(date_string)
    hover_popup.value = "Map updated."


def get_interactive_map(start_date=STARTDATE):
  m = get_choropleth(start_date)
  interact(update, new_date=date_picker)
  display(event_widget)
  return m

To use the map, click the "next event" button. It will take a moment to load in the new data, but you will be presented with a news headline, description, and link to the source of the story. The map will display the covid cases per 10 thousand residents in each county. The final date in the dataset is December 2nd. I encourage you to remember the stories presented and follow along how the map changes in the following weeks. COVID-19 has a latent period between infection and symptoms, and exponential spreading following a superspreader event takes a few weeks to manifest as well.

In [13]:
get_interactive_map()

Button(description='Next Event', style=ButtonStyle())

Button(description='Prev Event', style=ButtonStyle())

interactive(children=(DatePicker(value=datetime.date(2020, 1, 22), description='Date: ', disabled=True), Outpu…

HTML(value="<h1><b>First Travel-related Case of 2019 Novel Coronavirus Detected in United States.</b>:<br></h1…

Map(center=[48, -102], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

### Section 3: Analysis

In short, this map takes you through the growth and spread of COVID-19 across the United States. It is important to note that the color scale for the choropleth is relative, meaning for each date the deepest red is reserved for the county with the highest cases per capita. This means that when the map is redder overall, it suggests that COVID-19 is spreading evenly throughout the counties rather than being concentrated within hot spots. This is the trend that is observed overall: despite various prevention policies and measures designed to curb the spreading of COVID-19, time and again case spikes originate in singular locations before spreading outwards radially into neighboring regions. Super spreading events precede large shifts from blue to red in the general area they occurred in. States which roll back pandemic restrictions or reopen businesses while the virus’ spread remains uncontrolled see massive breakouts which do not cease to this day. The end result is an overwhelming sense of dread, scrolling along the timeline of the map. Dating back to as early as January, it was clear to informed newsreaders that the virus would be no joke. What followed has been nearly a full year of little collective action, and a nation that gets sicker and sicker with no clear end in sight.

Worst of all, what is never observed on this map are clear 1st and 2nd “waves” of coronavirus. Although many media headlines claim that the current record highs in case and death numbers are a “second wave” of the virus, the true reality is that COVID-19 never really stopped spreading. Any pauses in the acceleration of the virus were brief and quickly countered by new, larger exposures in other places. The map begins as a clean blue before COVID-19 hotspots appear like pimples on skin. These red splotches sometimes go away, as is the case with several of the early spikes which were swiftly dealt with and carefully managed by local health officials. In other cases, what initially appears as events localized to a single county spirals into a massive epidemic plaguing entire groups of states. What remains is a single, inevitable takeaway: this virus is a significant danger, and absolutely will not go away on its own. The only way to survive such exponential growth is to ensure the isolation and quarantine of every single exposure before the law of large numbers takes hold. But looking at the final few months of data, it seems that we may already be past the point of no return.


## References

Coronavirus data: https://github.com/CSSEGISandData/COVID-19

Dong, E., Du, H., &amp; Gardner, L. (2020, February 19). An interactive web-based dashboard to track COVID-19 in ... Retrieved December 2, 2020, from https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30120-1/fulltext

Aggregated data sources:

- Aggregated data sources:
  - World Health Organization (WHO): https://www.who.int/
  - European Centre for Disease Prevention and Control (ECDC): https://www.ecdc.europa.eu/en/geographical-distribution-2019-ncov-cases 
  - DXY.cn. Pneumonia. 2020. http://3g.dxy.cn/newh5/view/pneumonia
  - US CDC: https://www.cdc.gov/coronavirus/2019-ncov/index.html
  - BNO News: https://bnonews.com/index.php/2020/02/the-latest-coronavirus-cases/
  - WorldoMeters: https://www.worldometers.info/coronavirus/  
  - 1Point3Arces: https://coronavirus.1point3acres.com/en  
  - COVID Tracking Project: https://covidtracking.com/data. (US Testing and Hospitalization Data. We use the maximum reported value from "Currently" and "Cumulative" Hospitalized for our hospitalization number reported for each state.)
  - Los Angeles Times: https://www.latimes.com/projects/california-coronavirus-cases-tracking-outbreak/
  - The Mercury News: https://www.mercurynews.com/tag/coronavirus/

- US data sources at the state (Admin1) or county/city (Admin2) level:  
  - Washington State Department of Health: https://www.doh.wa.gov/emergencies/coronavirus
  - Maryland Department of Health: https://coronavirus.maryland.gov/
  - New York State Department of Health: https://health.data.ny.gov/Health/New-York-State-Statewide-COVID-19-Testing/xdss-u53e/data
  - New York City Health Department: https://www1.nyc.gov/site/doh/covid/covid-19-data.page
  - NYC Department of Health and Mental Hygiene: https://www1.nyc.gov/site/doh/covid/covid-19-data.page and https://github.com/nychealth/coronavirus-data
  - Florida Department of Health Dashboard: https://services1.arcgis.com/CY1LXxl9zlJeBuRZ/arcgis/rest/services/Florida_COVID19_Cases/FeatureServer/0
    and https://fdoh.maps.arcgis.com/apps/opsdashboard/index.html#/8d0de33f260d444c852a615dc7837c86
  - Colorado: https://covid19.colorado.gov/covid-19-data
  - Virginia: https://www.vdh.virginia.gov/coronavirus/
  - Northern Mariana Islands CNMI Department of Public Health: https://cnmichcc.maps.arcgis.com/apps/opsdashboard/index.html#/4061b674fc964efe84f7774b7979d2b5
  - Missouri Department of Health: https://www.arcgis.com/apps/MapSeries/index.html?appid=8e01a5d8d8bd4b4f85add006f9e14a9d
  - Missouri: Nodaway County: https://www.nodawaypublichealth.org/
  - Missouri: St. Louis City Department of Health: https://www.stlouis-mo.gov/covid-19/data/#totalsByDate
  - Missouri: St. Louis County: https://stlcorona.com/resources/covid-19-statistics1/
  - Massachusetts: https://www.mass.gov/info-details/covid-19-response-reporting
  - Michigan: https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173---,00.html 
  - Illinois Department of Public Health: https://dph.illinois.gov/covid19
  - Indiana State Department of Health: https://hub.mph.in.gov/dataset?q=COVID
  - Connecticut Department of Public Health: https://data.ct.gov/stories/s/COVID-19-data/wa3g-tfvc/
  - Ohio Department of Health: https://coronavirus.ohio.gov/wps/portal/gov/covid-19/home
  - Oregon Office of Emergency Management: https://experience.arcgis.com/experience/fff9f83827c5461583cd014fdf4587de
  - Tennessee Department of Health: https://www.tn.gov/health/cedep/ncov.html
  - Rhode Island Department of Health: https://ri-department-of-health-covid-19-data-rihealth.hub.arcgis.com/
  - Wisconsin Department of Health Services: https://www.dhs.wisconsin.gov/covid-19/data.htm
  - North Carolina City of Greenville GIS: https://www.arcgis.com/apps/opsdashboard/index.html#/7aeac695cafa4065ba1505b1cfa72747
  - Iowa State Government: https://coronavirus.iowa.gov/
  - Minnesota Department of Health: https://www.health.state.mn.us/diseases/coronavirus/situation.html
  - Alabama Public Health: https://www.alabamapublichealth.gov/covid19/
  - Mississippi State Department of Health: https://msdh.ms.gov/msdhsite/_static/14,0,420.html
  - Nebraska Department of Health and Human Services: https://experience.arcgis.com/experience/ece0db09da4d4ca68252c3967aa1e9dd
  - South Carolina Department of Health and Environmental Control: https://scdhec.gov/infectious-diseases/viruses/coronavirus-disease-2019-covid-19/sc-testing-data-projections-covid-19
  - Nevada Department of Health and Human Services: https://nvhealthresponse.nv.gov/
  - New Jersey Department of Health: https://covid19.nj.gov/
  - Delaware Emergency Management Agency: https://coronavirus.delaware.gov/
  - Utah Department of Health: https://coronavirus-dashboard.utah.gov/#overview
  - Arizona Emergency Support Function (ESF)/Recovery Support Function (RSF) Operations Dashboard: https://www.arcgis.com/apps/MapSeries/index.html?appid=62e6bfa682a34e6aae9d9255ac865467
  - Departamento de Salud, Puerto Rico: https://bioseguridad.maps.arcgis.com/apps/opsdashboard/index.html#/d7308c1abb4747e584329adf1215125e
  - Arkansas Department of Health: https://experience.arcgis.com/experience/c2ef4a4fcbe5458fbf2e48a21e4fece9
  - Maine Department of Health and Human Services: https://www.maine.gov/dhhs/mecdc/infectious-disease/epi/airborne/coronavirus/data.shtml
  - Pennsylvania Department of Health: https://www.health.pa.gov/topics/disease/coronavirus/Pages/Cases.aspx
  - City of Philadelphia: https://www.phila.gov/programs/coronavirus-disease-2019-covid-19/
  - Lancaster County: https://covid-19-lancastercountypa.hub.arcgis.com/
  - Chester County: https://chesco.maps.arcgis.com/apps/opsdashboard/index.html#/975082d579454c3ca7877db0a44e61ca
  - Louisiana Department of Health: http://ldh.la.gov/coronavirus/
  - Government of The District of Columbia: https://coronavirus.dc.gov/page/coronavirus-data 
  - North Dakota Department of Health: https://www.health.nd.gov/diseases-conditions/coronavirus/north-dakota-coronavirus-cases
  - Oklahoma State Department of Health: https://looker-dashboards.ok.gov/embed/dashboards/44
  - Guam Department of Public Health and Social Services: http://dphss.guam.gov/covid-19/ 
  - New Mexico Department of Health: https://cvprovider.nmhealth.org/public-dashboard.html
  - Kentucky Department of Public Health: https://kygeonet.maps.arcgis.com/apps/opsdashboard/index.html#/543ac64bc40445918cf8bc34dc40e334
  - Georgia Department of Public Health: https://dph.georgia.gov/covid-19-daily-status-report
  - State of Hawai'i Department of Health: https://health.hawaii.gov/coronavirusdisease2019/what-you-should-know/current-situation-in-hawaii/
  - Reno County Health Department: http://reno.maps.arcgis.com/apps/opsdashboard/index.html#/dfaef27aede1414b89abf393b2ccb994
  - Texas Department of State Health Services: https://dshs.texas.gov/coronavirus/
  - Texas: City of San Antonio: https://covid19.sanantonio.gov/Home
  - Texas: Brazioria County: https://www.brazoriacountytx.gov/departments/health-department/brazoria-county-coronavirus-map
  - Texas: Brazos County Health District: http://www.brazoshealth.org
  - Texas: Cameron County Public Health: https://www.cameroncounty.us/publichealth/index.php/coronavirus/
  - Texas: Collin County: https://www.collincountytx.gov/healthcare_services/Pages/news.aspx
  - Texas: Dallas County: https://www.dallascounty.org/departments/dchhs/2019-novel-coronavirus.php
  - Texas: Denton County: https://gis-covid19-dentoncounty.hub.arcgis.com/pages/covid-19cases
  - Texas: Ector County:  http://www.co.ector.tx.us/page/ector.CoronavirusCOVID19
  - Texas: City of El Paso: http://epstrong.org/results.php
  - Texas: Fayette County: https://www.co.fayette.tx.us/
  - Texas: Fort Bend County Health & Human Services: https://www.fbchealth.org/ncov/
  - Texas: Galveston County Health District: https://www.gchd.org/about-us/news-and-events/coronavirus-disease-2019-covid-19/galveston-county-confirmed-covid-19-cases
  - Texas: Harris County: https://publichealth.harriscountytx.gov/Resources/2019-Novel-Coronavirus
  - Texas: Hays County: https://hayscountytx.com/covid-19-information-for-hays-county-residents/
  - Texas: Hidalgo County Health and Human Services: https://www.hidalgocounty.us/2630/Coronavirus-Updates
  - Texas: Midland County: https://www.midlandtexas.gov/955/Coronavirus-COVID-19
  - Texas: Montgomery County: https://coronavirus-response-moco.hub.arcgis.com/
  - Texas: City of Corpus Christi: https://www.cctexas.com/coronavirus
  - Texas: Amarillo Public Health Department: https://covid-data-amarillo.hub.arcgis.com/
  - Texas: Tarrant County Public Health: https://www.tarrantcounty.com/en/public-health/disease-control---prevention/coronaviruas.html
  - Texas: City of Mount Pleasant: https://www.mpcity.net/632/COVID-19-UPDATES
  - Texas: City of San Angelo: https://www.cosatx.us/departments-services/health-services/coronavirus-covid-19#ad-image-0
  - Texas: San Angelo Standard-Times: https://www.gosanangelo.com/
  - Texas: Travis County: https://www.traviscountytx.gov/news/2020/1945-novel-coronavirus-covid-19-information
  - Texas: City of Laredo: https://www.cityoflaredo.com/coronavirus/
  - Texas: Williamson County & Cities Health District: http://www.wcchd.org/COVID-19/dashboard.php
  - California Department of Publich Health: https://www.cdph.ca.gov/covid19
  - California: Mariposa County: https://www.mariposacounty.org/1592/COVID-19-Information
  - California: Alameda County Public Health Department: https://covid-19.acgov.org/
  - California: Fresno County Public Health Department: https://www.co.fresno.ca.us/departments/public-health/covid-19
  - California: Humboldt County: https://humboldtgov.org/
  - California: Madera County: https://www.maderacounty.com/government/public-health/corona-virus-covid-19/covid-revised
  - California: Marin County Health & Human Services: https://coronavirus.marinhhs.org/
  - California: Orange County Health Care Agency: https://occovid19.ochealthinfo.com/coronavirus-in-oc
  - California: Placer County: https://www.placer.ca.gov/coronavirus
  - California: Riverside County: https://www.rivcoph.org/coronavirus
  - California: Sacramento County: https://www.saccounty.net/COVID-19/
  - California: San Francisco Department of Public Helath: https://www.sfdph.org/dph/alerts/coronavirus.asp
  - California: San Benito County Health & Human Services: https://hhsa.cosb.us/publichealth/communicable-disease/coronavirus/
  - California: San Joaquin County Public Health Services: http://www.sjcphs.org/coronavirus.aspx
  - California: San Mateo County: https://www.smchealth.org/coronavirus
  - California: Santa Cruz County Health Services Agency: http://www.santacruzhealth.org/HSAHome/HSADivisions/PublicHealth/CommunicableDiseaseControl/CoronavirusHome.aspx
  - California: Shasta County: https://www.co.shasta.ca.us/covid-19/overview
  - California: Solano County: https://www.co.shasta.ca.us/covid-19/overview
  - California: Sonoma County: https://socoemergency.org/emergency/novel-coronavirus/coronavirus-cases/
  - California: Stanislaus County Health Services Agency: http://schsa.org/publichealth/pages/corona-virus/
  - California: Ventura County: https://www.venturacountyrecovers.org/
  - California: Yolo County: https://www.yolocounty.org/health-human-services/adults/communicable-disease-investigation-and-control/novel-coronavirus-2019/
  - California: Los Angeles County: http://publichealth.lacounty.gov/media/coronavirus/
  - California: San Diego County: https://www.sandiegocounty.gov/coronavirus.html
  - California: Santa Clara County: https://www.sccgov.org/sites/covid19/
  - California: Imperial County Public Health Department: http://www.icphd.org/health-information-and-resources/healthy-facts/covid-19/
  - California: San Bernardino County: https://sbcovid19.com/
  - Montana Department of Public Health and Human Services: https://dphhs.mt.gov/publichealth/cdepi/diseases/coronavirusmt
  - South Dakota Department of Health: https://doh.sd.gov/news/coronavirus.aspx
  - Wyoming Department of Health: https://health.wyo.gov/publichealth/infectious-disease-epidemiology-unit/disease/novel-coronavirus/
  - New Hampshire Department of Health and Human Services: https://www.nh.gov/covid19/dashboard/summary.htm
  - Idaho Government: https://coronavirus.idaho.gov/
  - Virgin Islands Department of Health: https://www.covid19usvi.com/
  - Vermont Department of Health: https://www.healthvermont.gov/response/coronavirus-covid-19/current-activity-vermont
  - Kansas: Reno County Health Department: https://experience.arcgis.com/experience/9a7d44773e4c4a48b3e09e4d8673961b/page/page_18/
  - Kansas: Kansas Department Of Health And Environment: https://www.coronavirus.kdheks.gov/160/COVID-19-in-Kansas
  
  
  
Counties GeoJSON with FIPS: Plotly https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json


Population Data: US Census https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html#par_textimage


News Articles:


- https://www.cdc.gov/media/releases/2020/p0121-novel-coronavirus-travel-case.html
- https://www.ajmc.com/view/what-were-reading-global-health-emergency-declared-jj-pay-in-pelvic-mesh-suit-ban-on-fentanyl-lookalikes
- https://www.jurist.org/news/2020/02/trump-administration-declares-coronavirus-a-public-health-emergency-imposes-quarantine/
- https://www.cdc.gov/media/releases/2020/t0225-cdc-telebriefing-covid-19.html
- https://www.nytimes.com/2020/12/11/us/biogen-conference-covid-spread.html
- https://abc7news.com/coronavirus-princess-cruise-san-francisco-grand-placer-county-death/5988195/
- https://www.latimes.com/world-nation/story/2020-03-29/coronavirus-choir-outbreak
- https://apnews.com/article/52e12ca90c55b6e0c398d134a2cc286e
- https://travel.state.gov/content/travel/en/News/visas-news/presidential-proclamation-coronavirus.html
- https://www.gov.ca.gov/2020/03/19/governor-gavin-newsom-issues-stay-at-home-order/
- https://www.imperial.ac.uk/media/imperial-college/medicine/sph/ide/gida-fellowships/Imperial-College-COVID19-Exiting-Social-Distancing-24-03-2020.pdf
- https://www.aha.org/advisory/2020-03-25-senate-white-house-reach-agreement-relief-package-responding-covid-19
- https://www.whitehouse.gov/openingamerica/
- https://news.gallup.com/poll/309224/avoid-care-likely-covid-due-cost.aspx
- https://www.fda.gov/media/137564/download
- https://www.cnn.com/politics/live-news/fauci-testifies-coronavirus-05-12-2020/index.html
- https://www.cdc.gov/media/releases/2020/s0528-coronavirus-death-toll.html
- https://www.npr.org/sections/coronavirus-live-updates/2020/06/10/873473805/u-s-hits-2-million-coronavirus-cases-as-many-states-see-a-surge-of-patients?utm_campaign
- https://www.statnews.com/2020/06/30/u-s-could-see-100000-new-covid-19-cases-per-day-fauci-says/
- https://www.reuters.com/article/us-health-coronavirus-usa/reopenings-stall-as-u-s-records-nearly-50000-cases-of-covid-19-in-single-day-idUSKBN2426LN
- https://apnews.com/article/9dc4077f95d183649ca24a32a18abf01
- https://www.hhs.gov/sites/default/files/covid-19-faqs-hospitals-hospital-laboratory-acute-care-facility-data-reporting.pdf
- https://www.fda.gov/news-events/press-announcements/coronavirus-covid-19-update-daily-roundup-july-27-2020
- https://www.ajmc.com/view/a-timeline-of-covid19-developments-in-2020
- https://www.nytimes.com/2020/08/25/health/covid-19-testing-cdc.html
- https://www.washingtonpost.com/health/2020/10/17/sturgis-rally-spread/
- https://news.yahoo.com/exclusive-the-white-house-orders-end-to-covid-19-airport-screenings-for-international-travelers-172713942.html?guccounter=1&guce_referrer=aHR0cHM6Ly93d3cuYWptYy5jb20v&guce_referrer_sig=AQAAAFvRIi0-Frsg29RIAPMT5Cri0JngJQ5vy_sVm95rmTAKyfzQW8_riuVi_lANsEEbfBlRfjUZsyHq5VnBHspiQJ4nYrh67IspeWGxx5MRGBuGVms6pjhoxyIi84xxoTNZstl22EHn7rne93lJmMEjhXwDBphgDK5ApLYcTV7Hqkvf
- https://www.reuters.com/article/us-health-coronavirus-usa-cases-idUSKCN26G053?
- https://www.newsweek.com/amy-coney-barrett-rose-garden-event-was-wh-covid-superspreader-new-data-suggests-1537865
- https://www.npr.org/sections/latest-updates-trump-covid-19-results
- https://www.nejm.org/doi/full/10.1056/NEJMe2029812
- https://nypost.com/2020/11/11/this-couples-wedding-turned-into-a-covid-19-superspreader-event/
- https://www.nytimes.com/live/2020/11/04/world/covid-19-coronavirus-updates?
- https://www.politico.com/news/2020/11/19/cdc-thanksgiving-travel-coronavirus-438283?
- https://www.cnn.com/world/live-news/coronavirus-pandemic-12-02-20-intl/h_2cbfc56673b02f95ce59962dbf8888a0
